In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


***Step 1***

In this step, I created the Gold dimension tables by keeping only the current records from the Silver dimensions. This gives a clean, up-to-date view of customers, sales reps, products, geo, and country, making the data simple and ready to use for reporting in Power BI.


In [2]:
# Current rows only (conformed dims)
spark.table("silver_dim_customer_scd2").filter("is_current = 1") \
    .write.mode("overwrite").option("overwriteSchema","true").saveAsTable("gold_dim_customer")

spark.table("silver_dim_salesrep_scd2").filter("is_current = 1") \
    .write.mode("overwrite").option("overwriteSchema","true").saveAsTable("gold_dim_salesrep")

spark.table("silver_dim_product_scd2").filter("is_current = 1") \
    .write.mode("overwrite").option("overwriteSchema","true").saveAsTable("gold_dim_product")

spark.table("silver_dim_geo_scd2").filter("is_current = 1") \
    .write.mode("overwrite").option("overwriteSchema","true").saveAsTable("gold_dim_geo")

# Reference country (no SCD)
spark.table("silver_dim_country") \
    .write.mode("overwrite").option("overwriteSchema","true").saveAsTable("gold_dim_country")


StatementMeta(, 6b8bea08-607e-49a2-adbb-6bf9a111325b, 4, Finished, Available, Finished)

***Step 2. Create Gold fact table shells***

In this step, I created the Gold fact tables for sales and rebates, linking them to dimension keys and adding clean, business-ready fields like sales in USD and rebate details for easy reporting in Power BI.

In [3]:
spark.sql("""
CREATE TABLE IF NOT EXISTS gold_fact_sales (
  sale_transaction_id STRING,
  customer_sk BIGINT,
  salesrep_sk BIGINT,
  product_sk BIGINT,
  geo_sk BIGINT,
  unit_quantity INT,
  sale_amount DECIMAL(18,2),
  sale_amount_usd DECIMAL(18,2),
  sale_datetime TIMESTAMP,
  sale_date DATE,
  currencycode STRING,
  is_cross_currency INT
)
""")

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_fact_rebates (
  rebate_transaction_id STRING,
  sale_transaction_id STRING,
  customer_sk BIGINT,
  salesrep_sk BIGINT,
  rebate_type STRING,
  rebate_amount DECIMAL(18,2),
  rebate_datetime TIMESTAMP,
  rebate_date DATE,
  currencycode STRING
)
""")


StatementMeta(, 6b8bea08-607e-49a2-adbb-6bf9a111325b, 5, Finished, Available, Finished)

DataFrame[]

***Step 3. Build Gold Fact Sales (with SCD2 “as-of” joins)***

In this step, I built the Gold sales fact table by joining the Silver sales facts with the customer, sales rep, product, and geo dimensions. I matched records based on keys and valid date ranges to assign the correct surrogate keys. The final output is a clean, conformed sales fact table ready for reporting in Power BI.

In [4]:
from pyspark.sql import functions as F

# Load Silver dims
c = spark.table("silver_dim_customer_scd2").alias("c")
s = spark.table("silver_dim_salesrep_scd2").alias("s")
p = spark.table("silver_dim_product_scd2").alias("p")
g = spark.table("silver_dim_geo_scd2").alias("g")

# Load Silver fact
f = spark.table("silver_fact_sales_stage").alias("f")

gold_sales = (
    f
    .join(c,
          (F.col("f.customer_id") == F.col("c.customer_id")) &
          (F.col("f.sale_datetime").between(F.col("c.valid_from"), F.col("c.valid_to"))),
          "left")
    .join(s,
          (F.col("f.sales_rep_id") == F.col("s.sales_rep_id")) &
          (F.col("f.sale_datetime").between(F.col("s.valid_from"), F.col("s.valid_to"))),
          "left")
    .join(p,
          (F.col("f.product_code") == F.col("p.product_code")) &
          (F.col("f.sale_datetime").between(F.col("p.valid_from"), F.col("p.valid_to"))),
          "left")
    .join(g,
          (F.col("f.sales_country_code") == F.col("g.country_code")) &
          (
              (F.col("f.sale_datetime").between(F.col("g.valid_from"), F.col("g.valid_to")))
              | (F.col("g.is_current") == 1)
          ),
          "left")
    .select(
        F.col("f.sale_transaction_id"),
        F.col("c.customer_sk"),
        F.col("s.salesrep_sk"),
        F.col("p.product_sk"),
        F.col("g.geo_sk"),
        F.col("f.unit_quantity"),
        F.col("f.sale_amount"),
        F.col("f.sale_amount_usd"),
        F.col("f.sale_datetime"),
        F.col("f.sale_date"),
        F.col("f.currencycode"),
        F.col("f.is_cross_currency")
    )
)

gold_sales.write.mode("overwrite").insertInto("gold_fact_sales")


StatementMeta(, 6b8bea08-607e-49a2-adbb-6bf9a111325b, 6, Finished, Available, Finished)

***Step 4. Build Gold Fact Rebates (with SCD2 “as-of” joins)***

In this step, I created the Gold rebates fact table by joining the Silver rebates data with customer and sales rep dimensions. Using surrogate keys and valid date ranges, I ensured each rebate record is properly linked to its dimensions, producing a clean and conformed rebates fact table for reporting in Power BI.

In [6]:
# Load Silver Fact Rebates
r = spark.table("silver_fact_rebates_stage").alias("r")
gold_rebates = (
    r
    .join(c,
          (F.col("r.customer_id")==F.col("c.customer_id")) &
          (F.col("r.rebate_datetime").between(F.col("c.valid_from"), F.col("c.valid_to"))),
          "left")
    .join(s,
          (F.col("r.sales_rep_id")==F.col("s.sales_rep_id")) &
          (F.col("r.rebate_datetime").between(F.col("s.valid_from"), F.col("s.valid_to"))),
          "left")
    .select(
        F.col("r.rebate_transaction_id"),
        F.col("r.sale_transaction_id"),
        F.col("c.customer_sk"),
        F.col("s.salesrep_sk"),
        F.col("r.rebate_type"),
        F.col("r.rebate_amount"),
        F.col("r.rebate_datetime"),
        F.col("r.rebate_date"),
        F.col("r.currencycode")
    )
)

gold_rebates.write.mode("overwrite").insertInto("gold_fact_rebates")

StatementMeta(, 6b8bea08-607e-49a2-adbb-6bf9a111325b, 8, Finished, Available, Finished)

***Step 5. Aggregates***

***daily sales***

In this step, I created the daily sales aggregate table in the Gold layer by grouping sales data by date, geography, and currency. The table stores totals for sales amount, sales in USD, units sold, and distinct transactions, making it easier to analyze daily performance trends in Power BI.

In [4]:
from pyspark.sql import functions as F

daily_sales = (
    spark.table("gold_fact_sales")
    .groupBy("sale_date","geo_sk","currencycode")
    .agg(
        F.sum("sale_amount").alias("total_sales"),
        F.sum("sale_amount_usd").alias("total_sales_usd"),
        F.sum("unit_quantity").alias("total_units"),
        F.countDistinct("sale_transaction_id").alias("txn_id_count")
    )
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_agg_sales_daily (
  sale_date DATE,
  geo_sk BIGINT,
  currencycode STRING,
  total_sales DECIMAL(18,2),
  total_sales_usd DECIMAL(18,2),
  total_units BIGINT,
  txn_id_count BIGINT
)
""")

daily_sales.write.insertInto("gold_agg_sales_daily", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 6, Finished, Available, Finished)

*** Monthly sales***

In this step, I created the monthly sales aggregate table in the Gold layer by grouping sales data by year, month, geography, and currency. The table holds totals for sales amount, sales in USD, units sold, and distinct transactions, providing a clear view of monthly sales performance for reporting in Power BI.

In [5]:
monthly_sales = (
    spark.table("gold_fact_sales")
    .withColumn("year", F.year("sale_date"))
    .withColumn("month", F.month("sale_date"))
    .groupBy("year","month","geo_sk","currencycode")
    .agg(
        F.sum("sale_amount").alias("total_sales"),
        F.sum("sale_amount_usd").alias("total_sales_usd"),
        F.sum("unit_quantity").alias("total_units"),
        F.countDistinct("sale_transaction_id").alias("txn_id_count")
    )
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_agg_sales_monthly (
  year INT,
  month INT,
  geo_sk BIGINT,
  currencycode STRING,
  total_sales DECIMAL(18,2),
  total_sales_usd DECIMAL(18,2),
  total_units BIGINT,
  txn_id_count BIGINT
)
""")

monthly_sales.write.insertInto("gold_agg_sales_monthly", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 7, Finished, Available, Finished)

*** yearly sales***

In this step, I created the year-to-date (YTD) sales aggregate table in the Gold layer by grouping sales data by year, geography, and currency. The table stores totals for YTD sales amount, sales in USD, units sold, and unique transactions, helping track overall yearly performance in Power BI.

In [6]:
ytd_sales = (
    spark.table("gold_fact_sales")
    .withColumn("year", F.year("sale_date"))
    .groupBy("year","geo_sk","currencycode")
    .agg(
        F.sum("sale_amount").alias("ytd_sales"),
        F.sum("sale_amount_usd").alias("ytd_sales_usd"),
        F.sum("unit_quantity").alias("ytd_units"),
        F.countDistinct("sale_transaction_id").alias("txn_id_count")
    )
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_agg_sales_ytd (
  year INT,
  geo_sk BIGINT,
  currencycode STRING,
  ytd_sales DECIMAL(18,2),
  ytd_sales_usd DECIMAL(18,2),
  ytd_units BIGINT,
  txn_id_count BIGINT
)
""")

ytd_sales.write.insertInto("gold_agg_sales_ytd", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 8, Finished, Available, Finished)

*** customer sales aggregate***

In this step, I created the customer-level sales aggregate table in the Gold layer by grouping sales data by customer. The table records each customer’s total sales, sales in USD, units purchased, and distinct transactions, enabling customer-level performance analysis in Power BI.


In [13]:
customer_sales = (
    spark.table("gold_fact_sales")
    .groupBy("customer_sk")
    .agg(
        F.sum("sale_amount").alias("total_sales"),
        F.sum("sale_amount_usd").alias("total_sales_usd"),
        F.sum("unit_quantity").alias("total_units"),
        F.countDistinct("sale_transaction_id").alias("txn_id_count")
    )
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_agg_sales_customer (
  customer_sk BIGINT,
  total_sales DECIMAL(18,2),
  total_sales_usd DECIMAL(18,2),
  total_units BIGINT,
  txn_id_count BIGINT
)
""")

customer_sales.write.insertInto("gold_agg_sales_customer", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 15, Finished, Available, Finished)

***Step 6. KPI Tables***

***6a. Product KPI***

In this step, I created the product KPI table in the Gold layer by grouping sales data by product. The table captures each product’s total sales, total units sold, and number of unique transactions, providing key metrics for product-level performance reporting in Power BI.

In [11]:
product_kpi = (
    spark.table("gold_fact_sales")
    .groupBy("product_sk")
    .agg(
        F.sum("sale_amount").alias("total_sales"),
        F.sum("unit_quantity").alias("total_units"),
        F.countDistinct("sale_transaction_id").alias("txn_id_count")
    )
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_kpi_product (
  product_sk BIGINT,
  total_sales DECIMAL(18,2),
  total_units BIGINT,
  txn_id_count BIGINT
)
""")

product_kpi.write.insertInto("gold_kpi_product", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 13, Finished, Available, Finished)

***6b. Region KPI***

In this step, I created the region KPI table in the Gold layer by grouping sales data by geography. The table shows total sales, sales in USD, and the number of unique transactions for each region, making it easier to track regional performance in Power BI.

In [9]:
region_kpi = (
    spark.table("gold_fact_sales")
    .groupBy("geo_sk")
    .agg(
        F.sum("sale_amount").alias("total_sales"),
        F.sum("sale_amount_usd").alias("total_sales_usd"),
        F.countDistinct("sale_transaction_id").alias("txn_id_count")
    )
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_kpi_region (
  geo_sk BIGINT,
  total_sales DECIMAL(18,2),
  total_sales_usd DECIMAL(18,2),
  txn_id_count BIGINT
)
""")

region_kpi.write.insertInto("gold_kpi_region", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 11, Finished, Available, Finished)

***6c. SalesRep KPI***

In this step, I created the sales rep KPI table in the Gold layer by grouping sales data by sales representative. The table captures each rep’s total sales, total units sold, and number of unique transactions, helping measure sales rep performance in Power BI.

In [14]:
salesrep_kpi = (
    spark.table("gold_fact_sales")
    .groupBy("salesrep_sk")
    .agg(
        F.sum("sale_amount").alias("total_sales"),
        F.sum("unit_quantity").alias("total_units"),
        F.countDistinct("sale_transaction_id").alias("txn_id_count")
    )
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_kpi_salesrep (
  salesrep_sk BIGINT,
  total_sales DECIMAL(18,2),
  total_units BIGINT,
  txn_id_count BIGINT
)
""")

salesrep_kpi.write.insertInto("gold_kpi_salesrep", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 16, Finished, Available, Finished)

***6d. Cross-Currency KPI***

In this step, I created the cross-currency KPI table in the Gold layer by filtering sales where the currency did not match the country’s standard. The table tracks cross-currency sales amounts (both original and USD) and unique transactions by date and region, allowing analysis of currency mismatches in Power BI.

In [15]:
cross_currency_kpi = (
    spark.table("gold_fact_sales")
    .filter("is_cross_currency = 1")
    .groupBy("sale_date","geo_sk")
    .agg(
        F.sum("sale_amount").alias("cross_currency_sales"),
        F.sum("sale_amount_usd").alias("cross_currency_sales_usd"),
        F.countDistinct("sale_transaction_id").alias("txn_id_count")
    )
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_kpi_cross_currency (
  sale_date DATE,
  geo_sk BIGINT,
  cross_currency_sales DECIMAL(18,2),
  cross_currency_sales_usd DECIMAL(18,2),
  txn_id_count BIGINT
)
""")

cross_currency_kpi.write.insertInto("gold_kpi_cross_currency", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 17, Finished, Available, Finished)

***Step 7. Aggregate – Rebates Summary***

In this step, I created the rebates summary aggregate table in the Gold layer by grouping rebate data by year, month, customer, sales rep, and rebate type. The table records total rebate amounts and the number of rebate transactions, making it easier to analyze rebate trends and performance in Power BI.

In [16]:
from pyspark.sql import functions as F

rebates_summary = (
    spark.table("gold_fact_rebates")
    .withColumn("year", F.year("rebate_date"))
    .withColumn("month", F.month("rebate_date"))
    .groupBy("year","month","customer_sk","salesrep_sk","rebate_type")
    .agg(
        F.sum("rebate_amount").alias("total_rebate_amount"),
        F.countDistinct("rebate_transaction_id").alias("rebate_txn_id_count")
    )
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_agg_rebates_summary (
  year INT,
  month INT,
  customer_sk BIGINT,
  salesrep_sk BIGINT,
  rebate_type STRING,
  total_rebate_amount DECIMAL(18,2),
  rebate_txn_id_count BIGINT
)
""")


rebates_summary.write.insertInto("gold_agg_rebates_summary", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 18, Finished, Available, Finished)

***Step 8. KPI – Rebates vs Sales***

In this step, I created the rebates vs sales KPI table in the Gold layer by joining customer-level sales and rebate data. The table stores total sales (in USD), total rebate amounts, and the rebate-to-sales ratio, helping measure the impact of rebates on overall sales in Power BI.

In [17]:
rebates_vs_sales = (
    spark.table("gold_fact_sales").groupBy("customer_sk")
    .agg(
        F.sum("sale_amount_usd").alias("total_sales_usd")
    )
    .join(
        spark.table("gold_fact_rebates").groupBy("customer_sk")
        .agg(
            F.sum("rebate_amount").alias("total_rebate_amount")
        ),
        "customer_sk", "left"
    )
    .withColumn("rebate_ratio", 
                (F.col("total_rebate_amount")/F.col("total_sales_usd"))*100)
)

spark.sql("""
CREATE TABLE IF NOT EXISTS gold_kpi_rebates_vs_sales (
  customer_sk BIGINT,
  total_sales_usd DECIMAL(18,2),
  total_rebate_amount DECIMAL(18,2),
  rebate_ratio DECIMAL(18,2)
)
""")


rebates_vs_sales.write.insertInto("gold_kpi_rebates_vs_sales", overwrite=True)


StatementMeta(, 0dbd91f5-8db1-4dfa-83de-75e7c74aeb76, 19, Finished, Available, Finished)